# README

## WARNING
This method (CP-SAT) is very slow (about 30 minutes on M1 Pro). The mixed-integer programming (MIP) method took about 5 minutes.

This document can be run directly to solve the problem. It contains the key mathematical theory and corresponding code snippets. 

#### Load CP-SAT and itertools

In [1]:
from ortools.sat.python import cp_model
model = cp_model.CpModel()
solver = cp_model.CpSolver()

from itertools import combinations

#### Problem setup
**Setup problem indexing**
* Cells are indexed 1 to 28.
* Each almost magic square consists of 9 indices.
* slices to check the sums are pre-generated for convenience.

In [2]:
# indexes
#      1,  2,  3
#      4,  5,  6,  7,  8
#  9, 10, 11, 12, 13, 14
# 15, 16, 17, 18, 19, 20
# 21, 22, 23, 24, 25
#         26, 27, 28 
#
# Square 1
sq1 = [
     1, 2, 3,
     4, 5, 6,
    10,11,12]
sq2 = [
     6, 7, 8,
    12,13,14,
    18,19,20]
sq3 = [
     9,10,11,
    15,16,17,
    21,22,23]
sq4 = [
    17,18,19,
    23,24,25,
    26,27,28]
sqs = [sq1, sq2, sq3, sq4]

horzs = [slice(3*i, 3*i+3  ) for i in range(3)] # horizontal
verts = [slice(  i,     9,3) for i in range(3)] # vertical
diags = [slice(  0,     9,4), # main diag
         slice(  2,     7,2)] # off diag


#### Setup variables:
* $x_{n,v} = 1$ $\Leftrightarrow$ cell $n$ contains value $v$.

In [3]:
vmax = 1000 # don't bother looking higher than some number

## Define and save variables
nrange = [i+1 for i in range(28)]
vrange = [i+1 for i in range(vmax)] 
x = {}
for n in nrange:
    for v in vrange:
        x[n,v] = model.NewBoolVar(f'x[{n},{v}]')

#### Uniqueness constraint
* Each value $v$ can only appear at most once.
* Each cell must have a value $v$.

In [4]:
## Uniqueness
for v in vrange:
    model.Add(sum(x[n,v] for n in nrange) <= 1)
for n in nrange:
    model.Add(sum(x[n,v] for v in vrange) == 1)

### Sum constraints (for each almost magic square)
* For each square, we first create expressions for all horizontal, diagonal sums. 
* Given any two sum, the absolute value of their difference must be no more than 1. While the absolute value function is nonlinear, we can write the constraint piece-wise, i.e. in two parts:
    * $sum_1 - sum_2 \leq 1$
    * $sum_1 - sum_2 \geq -1$
* We should also not bother looking for worse solutions than the example one
    * $\sum_{n,v} v x_{n,v} \leq 1828$

In [5]:
## Sums constraints
for sq in sqs:
    sums = [sum(v*x[n,v] for n in sq[sum_slice] for v in vrange) for sum_slice in horzs + verts + diags]
    for sum1, sum2 in combinations(sums, 2):
        model.Add(sum1 - sum2 <=  1)
        model.Add(sum1 - sum2 >= -1)

model.Add(sum(v*x[n,v] for n in nrange for v in vrange) <= 1828)


#### Objective function
Minimize $\sum_n x_n$

In [8]:
model.Minimize(sum(v*x[n,v] for n in nrange for v in vrange))
status = solver.Solve(model)

^C pressed 1 times. Interrupting the solver. Press 3 times to force termination.


#### Store and save the solution

In [16]:
import numpy as np
def assignSol():
    X = np.zeros((len(nrange), len(vrange)), dtype=int)
    V = np.zeros((len(nrange)), dtype=int)
    for n in nrange:
        for v in vrange:
            if solver.BooleanValue(x[n,v]):
                X[n-1,v-1] = 1
                V[n-1] += v
    return V, X

In [17]:
V, X = assignSol()
print(V)
# [26,  2, 34, 29, 21, 13,  4, 18, 24,  7, 39, 16, 12,  6, 37, 23,  9,
#        5, 19, 10,  8, 40, 22, 11,  1,  3, 17, 14]

array([26,  2, 34, 29, 21, 13,  4, 18, 24,  7, 39, 16, 12,  6, 37, 23,  9,
        5, 19, 10,  8, 40, 22, 11,  1,  3, 17, 14])